In [1]:
import pandas as pd
from plydata import arrange, count, group_by, head, distinct, define, do, query

table=pd.read_csv("data/20170508-entities.csv")

# table.head()
# table >> head()

table.head()

,Unnamed: 0,confidence,dpaid,end,label,start,surface,tool,uri
0,0,0.949237,urn_newsml_dpa.com_20090101_170221-99-370286v-3,18,NaN,13,Lahti,ambiverse,http://www.wikidata.org/entity/Q28911
1,1,0.143735,urn_newsml_dpa.com_20090101_170221-99-370286v-3,185,NaN,176,deutschen,ambiverse,http://www.wikidata.org/entity/Q183
2,2,0.949237,urn_newsml_dpa.com_20090101_170221-99-370286v-3,203,NaN,198,Lahti,ambiverse,http://www.wikidata.org/entity/Q28911
3,3,1.000000,urn_newsml_dpa.com_20090101_170221-99-370286v-3,208,NaN,205,dpa,ambiverse,http://www.wikidata.org/entity/Q312653
4,4,0.929053,urn_newsml_dpa.com_20090101_170221-99-370286v-3,239,NaN,231,Finnland,ambiverse,http://www.wikidata.org/entity/Q33


# Tool-Vergleich

In [2]:
# tooltable=table.groupby("tool").count()
# tooltable.head(100)

table >> group_by("tool") >> count()

,tool,n
0,ambiverse,1515
1,dandelion,2475
2,txtwerk,2171


# Verschiedene Surfaceforms

In [3]:
# print("Disctinct Surfaceforms:",len(set(table["surface"])))

table >> define(surface="surface.map(lambda a: a.upper())")\
    >> distinct("surface",'first') \
    >> count()

,n
0,2065


# Ueberschneidungen zwischen den tools

In [4]:
# eine Funktion, die fuer jede Zeile ausgefuehrt wird und eine neue Spalte zurueckliefert 

# func=lambda a: a["dpaid"]+":"+str(a["start"])

# table["startpos"]=table.apply(func, axis=1)

# table["entitydef"]=table.apply(lambda a: a["tool"][0]+"="+a["surface"],axis=1)

#startpostable=table.groupby("startpos").aggregate({ "start"     : "count", 
#                                                    "tool"      : lambda a: ",".join(sorted(a)), 
#                                                    "surface"   : lambda a: ",".join(set(a)),
#                                                    "entitydef" : lambda a: ",".join(a)
#                                                  })

# startpostable.head()

def add_startpos(df) :
    df["startpos"]=df.apply(lambda a: a["dpaid"][-20:]+":"+str(a["start"]),axis=1)
    return df

def aggregate_surface(df) :
    sf=df.apply(lambda a: a["tool"][0]+":"+a["surface"],axis=1)
    return pd.DataFrame({'found' : [len(df)], 
                         'surfaces' : [",".join(sf)],
                         'distinct' : [len(set(df["surface"]))],
                         'tools' : [",".join(sorted(set(df["tool"])))] 
                        })
#>> do(startpos = lambda df : df.apply(lambda a: a["dpaid"][:-10]+":"+str(a["start"]),axis=1)) \
     

startpostable = table >> do(add_startpos) \
      >> group_by('startpos') \
      >> do(aggregate_surface) 
        
startpostable >> head()


,startpos,distinct,found,surfaces,tools
0,170223-99-408779v-10:0,1,3,"a:Fulda,d:Fulda,t:Fulda","ambiverse,dandelion,txtwerk"
1,170223-99-408779v-10:1038,1,1,a:Mail,ambiverse
2,170223-99-408779v-10:1107,1,2,"d:Sozialwissenschaftler,t:Sozialwissenschaftler","dandelion,txtwerk"
3,170223-99-408779v-10:1133,1,3,"a:Hochschule Fulda,d:Hochschule Fulda,t:Hochsc...","ambiverse,dandelion,txtwerk"
4,170223-99-408779v-10:1233,1,2,"d:Karnevalsverein,t:Karnevalsverein","dandelion,txtwerk"


In [178]:
#from itertools import combinations
#
#tools_list=sorted(set(table["tool"]))
#
#for n in range(1,1+len(tools_list)) :
#    for c in combinations(tools_list,n) :
#        tools=",".join(c) 
#        print(tools,":",len(startpostable.query("tools=='%s'" % tools)))
        
startpostable >> group_by("tools") >> count()

,tools,n
0,ambiverse,237
1,"ambiverse,dandelion",142
2,"ambiverse,dandelion,txtwerk",926
3,"ambiverse,txtwerk",210
4,dandelion,710
5,"dandelion,txtwerk",697
6,txtwerk,338


In [183]:
startpostable >> group_by('distinct','found','tools') >> count() 

,distinct,found,tools,n
0,1,1,ambiverse,237
1,1,1,dandelion,710
2,1,1,txtwerk,338
3,1,2,"ambiverse,dandelion",91
4,1,2,"ambiverse,txtwerk",194
5,1,2,"dandelion,txtwerk",673
6,1,3,"ambiverse,dandelion,txtwerk",811
7,2,2,"ambiverse,dandelion",51
8,2,2,"ambiverse,txtwerk",16
9,2,2,"dandelion,txtwerk",24
